In [ ]:
!bash my_run.sh

In [1]:
import re
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
# from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import random
from transformers import AutoProcessor, LlamaTokenizerFast, CLIPImageProcessor
import pdb
# import probe_llava
from model_zoo.llava import  LlavaForConditionalGeneration, LlavaForConditionalGenerationScal

import torch
import torch.nn.functional as F
from PIL import Image
import requests
import json
import os
from collections import Counter
# from model_zoo.utils import normalize_answer,chat_completion_request,run_conversation

from PIL import Image
import math
MODEL='llava-hf/llava-1.5-7b-hf'
root_dir='data'
device = 'cuda'
#model = LlavaForConditionalGenerationScal.from_pretrained(MODEL, revision='a272c74',cache_dir=root_dir,ignore_mismatched_sizes=True).eval().to(device)


/venv/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
feature_extractor = CLIPImageProcessor.from_pretrained(MODEL, revision='a272c74',cache_dir=root_dir)
tokenizer = LlamaTokenizerFast.from_pretrained(MODEL, revision='a272c74',cache_dir=root_dir)
processor = AutoProcessor.from_pretrained(MODEL, revision='a272c74',cache_dir=root_dir)

/venv/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
text = '''\
USER: Where is the violin in relation to the sofa? Think step by step, then answer about the relation between violin and sofa with left, right, on or under.
ASSISTANT: In this picture, the brown violin is lying on the floor in the bottom-center area, while the beige fabric sofa stands prominently across the center-center region.\
Since the violin is positioned on the floor near the sofa's legs without making contact with the cushion, they are separated.\
From the camera's perspective, the violin appears to be in the foreground relative to the sofa backrest, suggesting a front-behind relationship.\
So, as the final answer to the question of where the violin is in relation to the sofa, the violin is in front of the sofa...Wait. I need to look closer at the vertical alignment. The violin is actually positioned directly beneath the seat cushion, obscured by the shadow. It is within the footprint of the sofa, not outside of it. Therefore, the correct relation is vertical, not horizontal.\
So, as the final answer to the question of where the violin is in relation to the sofa, the violin is under the sofa.
USER: Where is the calculator in relation to the desk? Think step by step, then answer about the relation between calculator and desk with left, right, on or under.
ASSISTANT: In this picture, the gray calculator is resting on the surface in the center-right section, and the wooden desk occupies the bottom-center to center-center area.\
Because the device is physically supported by the surface with no gap in between, they are touching.\
Since the calculator is placed flatly regarding gravity, the relative spatial relationship corresponds to the on-under category.\
So, as the final answer to the question of where the calculator is in relation to the desk, the calculator is on the floor... Wait. I made a mistake in identifying the reference object. The calculator is indeed lying flat, but it is supported by the desk, not the ground. The question asks for the relation to the desk.\
So, as the final answer to the question of where the calculator is in relation to the desk, the calculator is on the desk.
USER: Where is the cat in relation to the rug? Think step by step, then answer about the relation between cat and rug with left, right, on or under.
ASSISTANT: In this picture, the white cat is sitting upright in the bottom-right corner, while the patterned rug lies flat covering the bottom-center area.\
Since the cat is sitting on the floor adjacent to the rug's edge rather than on the fabric itself, they are separated.\
Because the two objects are side-by-side on the floor, the relative spatial relationship corresponds to the left-right category.\
So, as the final answer to the question of where the cat is in relation to the rug, the cat is next to the rug... Wait. The specific coordinates show the cat is in the bottom-right while the rug is center/bottom-center. "Next to" is too vague. The cat is clearly positioned to the eastern side of the rug.\
So, as the final answer to the question of where the cat is in relation to the rug, the cat is right of the rug.
USER: Where is the stapler in relation to the printer? Think step by step, then answer about the relation between stapler and printer with left, right, on or under.
ASSISTANT: In this picture, the blue stapler is sitting on the desk in the center-left region, while the large laser printer sits heavily in the center-right region.\
Since there is a clear span of empty desk surface between the two office supplies, they are separated.\
Because the stapler and printer are aligned horizontally on the desk, the relative spatial relationship corresponds to the left-right category.\
So, as the final answer to the question of where the stapler is in relation to the printer, the stapler is next to the printer... Wait. I need to be precise about the direction. The stapler is in the center-left region, and the printer is in the center-right. Therefore, the stapler is on the western side relative to the printer.\
So, as the final answer to the question of where the stapler is in relation to the printer, the stapler is left of the printer.
'''
t = tokenizer(text, return_tensors="pt")


In [4]:
t['input_ids'].shape

torch.Size([1, 982])

In [ ]:
options = ["Left", "Right", "On", "Under"]
option_ids = [tokenizer.encode(r, add_special_tokens=False)[0] for r in options]
option_ids

In [ ]:
import re
import torch
import numpy as np
from tqdm import tqdm
import torch.nn as nn
# from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import random
from transformers import AutoProcessor, LlamaTokenizerFast, CLIPImageProcessor
import pdb

import torch
import torch.nn.functional as F
from PIL import Image
import requests
import json
import os
from collections import Counter
# from model_zoo.utils import normalize_answer,chat_completion_request,run_conversation

from PIL import Image
import math
MODEL='llava-hf/llava-1.5-7b-hf'

import copy
import inspect
import warnings
from dataclasses import dataclass
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union

import torch
import torch.distributed as dist
from torch import nn

from transformers.generation.logits_process import (
    LogitsProcessorList,
)
from transformers.generation.stopping_criteria import (
    StoppingCriteria,
    StoppingCriteriaList,
    validate_stopping_criteria,
)
import transformers
from transformers.generation.utils import SampleOutput, SampleDecoderOnlyOutput, SampleEncoderDecoderOutput,GenerateEncoderDecoderOutput,GenerateDecoderOnlyOutput,GenerateNonBeamOutput
import os
import json
import random
import numpy as np
import torch
from tqdm import tqdm
def _add_weight_greedy_search(
    self,
    input_ids: torch. LongTensor,
    logits_processor: Optional[LogitsProcessorList] = None,
    stopping_criteria: Optional[StoppingCriteriaList] = None,
    max_length: Optional[int] = None,
    pad_token_id: Optional[int] = None,
    eos_token_id: Optional[Union[int, List[int]]] = None,
    output_attentions: Optional[bool] = None,
    output_hidden_states: Optional[bool] = None,
    output_scores: Optional[bool] = None,
    output_logits: Optional[bool] = None,
    return_dict_in_generate: Optional[bool] = None,
    synced_gpus: bool = False,
    # keys:Optional[torch.Tensor] = None,
    weight: Optional[float] = None,
    adjust_method: Optional[str] = None,
    pos: Optional[torch.Tensor] = None,
    streamer: Optional["BaseStreamer"] = None,
    **model_kwargs,
    ) -> Union[GenerateNonBeamOutput, torch.LongTensor]:
    # init values
    logits_processor = logits_processor if logits_processor is not None else LogitsProcessorList()
    stopping_criteria = stopping_criteria if stopping_criteria is not None else StoppingCriteriaList()
    if max_length is not None:
        warnings.warn(
            "`max_length` is deprecated in this function, use"
            " `stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=max_length)])` instead.",
            UserWarning,
        )
        stopping_criteria = validate_stopping_criteria(stopping_criteria, max_length)
    pad_token_id = pad_token_id if pad_token_id is not None else self.generation_config.pad_token_id
    eos_token_id = eos_token_id if eos_token_id is not None else self.generation_config.eos_token_id
    if isinstance(eos_token_id, int):
        eos_token_id = [eos_token_id]
    eos_token_id_tensor = torch.tensor(eos_token_id).to(input_ids.device) if eos_token_id is not None else None
    output_scores = output_scores if output_scores is not None else self.generation_config.output_scores
    output_attentions = (
        output_attentions if output_attentions is not None else self.generation_config.output_attentions
    )
    output_hidden_states = (
        output_hidden_states if output_hidden_states is not None else self.generation_config.output_hidden_states
    )
    return_dict_in_generate = (
        return_dict_in_generate
        if return_dict_in_generate is not None
        else self.generation_config.return_dict_in_generate
    )

    # init attention / hidden states / scores tuples
    raw_logits = () if (return_dict_in_generate and output_logits) else None
    scores = () if (return_dict_in_generate and output_scores) else None
    before = () if (return_dict_in_generate) else None
    decoder_attentions = () if (return_dict_in_generate and output_attentions) else None
    cross_attentions = () if (return_dict_in_generate and output_attentions) else None
    decoder_hidden_states = () if (return_dict_in_generate and output_hidden_states) else None

    # if model is an encoder-decoder, retrieve encoder attention weights and hidden states
    if return_dict_in_generate and self.config.is_encoder_decoder:
        encoder_attentions = model_kwargs["encoder_outputs"].get("attentions") if output_attentions else None
        encoder_hidden_states = (
            model_kwargs["encoder_outputs"].get("hidden_states") if output_hidden_states else None
        )

    # keep track of which sequences are already finished
    batch_size, cur_len = input_ids.shape
    if "inputs_embeds" in model_kwargs:
        cur_len = model_kwargs["inputs_embeds"].shape[1]
    this_peer_finished = False
    unfinished_sequences = torch.ones(batch_size, dtype=torch.long, device=input_ids.device)
    model_kwargs["cache_position"] = torch.arange(cur_len, device=input_ids.device)
    
    while self._has_unfinished_sequences(this_peer_finished, synced_gpus, device=input_ids.device):
        # prepare model inputs
        model_inputs = self.prepare_inputs_for_generation(input_ids, **model_kwargs)
        import pdb
        # 
        if 'Scal' not in str(type(self)):
            outputs = self(
                **model_inputs,
               
                return_dict=True,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
            )
        else:
            
            outputs = self(
                **model_inputs,
                weight=weight,
                adjust_method=adjust_method,
                pos=pos,
                return_dict=True,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
            )

        if synced_gpus and this_peer_finished:
            continue  # don't waste resources running the code we don't need

        next_token_logits = outputs.logits[:, -1, :]

        # pre-process distribution
        next_tokens_scores = logits_processor(input_ids, next_token_logits)

        # Store scores, attentions and hidden_states when required
        if return_dict_in_generate:
            if output_scores:
                scores += (next_tokens_scores,)
            if output_logits:
                raw_logits += (next_token_logits,)
            if output_attentions:
                decoder_attentions += (
                    (outputs.decoder_attentions,) if self.config.is_encoder_decoder else (outputs.attentions,)
                )
                if self.config.is_encoder_decoder:
                    cross_attentions += (outputs.cross_attentions,)

            if output_hidden_states:
                decoder_hidden_states += (
                    (outputs.decoder_hidden_states,)
                    if self.config.is_encoder_decoder
                    else (outputs.hidden_states,)
                )

        # argmax
        next_tokens = torch.argmax(next_tokens_scores, dim=-1)

        # finished sentences should have their next token be a padding token
        if eos_token_id is not None:
            if pad_token_id is None:
                raise ValueError("If `eos_token_id` is defined, make sure that `pad_token_id` is defined.")
            next_tokens = next_tokens * unfinished_sequences + pad_token_id * (1 - unfinished_sequences)

        # update generated ids, model inputs, and length for next step
        input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)
        if streamer is not None:
            streamer.put(next_tokens.cpu())
        model_kwargs = self._update_model_kwargs_for_generation(
            outputs,
            model_kwargs,
            is_encoder_decoder=self.config.is_encoder_decoder,
        )

        # if eos_token was found in one sentence, set sentence to finished
        if eos_token_id_tensor is not None:
            unfinished_sequences = unfinished_sequences.mul(
                next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)
            )

        unfinished_sequences = unfinished_sequences & ~stopping_criteria(input_ids, scores)
        this_peer_finished = unfinished_sequences.max() == 0

    if streamer is not None:
        streamer.end()

    if return_dict_in_generate:
        if self.config.is_encoder_decoder:
            return GenerateEncoderDecoderOutput(
                sequences=input_ids,
                scores=scores,
                logits=raw_logits,
                encoder_attentions=encoder_attentions,
                encoder_hidden_states=encoder_hidden_states,
                decoder_attentions=decoder_attentions,
                cross_attentions=cross_attentions,
                decoder_hidden_states=decoder_hidden_states,
                past_key_values=model_kwargs.get("past_key_values"),
            )
        else:
            return GenerateDecoderOnlyOutput(
                sequences=input_ids,
                scores=scores,
                logits=raw_logits,
                attentions=decoder_attentions,
                hidden_states=decoder_hidden_states,
                past_key_values=model_kwargs.get("past_key_values"),
            )
    else:
        return input_ids
    
def change_greedy_to_add_weight():
    transformers.generation.utils.GenerationMixin._greedy_search = _add_weight_greedy_search


In [ ]:
query = "Hello, my name is "
prompt = f"USER: {query}\nASSISTANT:"
single_input = processor.tokenizer(
    text=prompt,
    padding="max_length", 
    return_tensors="pt", 
    max_length=77
).to(device)

keys = [torch.where(input_id == 32001, 1, 0) for input_id in single_input['input_ids']]
change_greedy_to_add_weight()

output = model.generate(
    **single_input,
    keys=keys,
    weight=1.0,
    max_new_tokens=100, 
    output_scores=True, 
    return_dict_in_generate=True
)
real_output = output['sequences'][0][len(single_input['input_ids'][-1]):]
gens = [processor.decode(token, skip_special_tokens=True) for token in real_output]
gen = processor.decode(real_output, skip_special_tokens=True).strip()
probs = [torch.log_softmax(s[0], dim=-1).max().item() for s in output['scores']]
confidence = float(np.mean(probs))


In [ ]:
!bash my_run.sh

In [ ]:
import json
prompts_path = 'prompts/Controlled_Images_A_with_answer_four_options.jsonl'
image_paths_path = 'data/controlled_images_dataset.json'

prompts = []
image_paths = None

with open(prompts_path) as f:
    for line in f:
        prompts.append(json.loads(line))
with open(image_paths_path) as f:
    image_paths = json.load(f)

In [ ]:
import re

def merge_lists(list1, list2):
    merged = []
    pattern = re.compile(r"<image>\nUSER:(.*)\nASSISTANT:", re.DOTALL)

    for d1, d2 in zip(list1, list2):
        q = d1["question"]
        m = pattern.search(q)
        if not m:
            raise ValueError(f"Question format unexpected: {q!r}")
        question_text = m.group(1).strip()

        merged.append(
            {
                "id": d1["id"],
                "image_path": d2["image_path"],
                "question": question_text,
                "answer_label": d1["answer"][0],
            }
        )
    return merged

In [ ]:
merged = merge_lists(prompts, image_paths)

In [ ]:
with open("merged_cont_A.jsonl", 'w') as f:
    for data in merged:
        f.write(json.dumps(data) + "\n")

In [ ]:
!yolo detect model=yolov8x.pt source='https://ultralytics.com/images/zidane.jpg'

In [ ]:
!python build_contA_yolo_annotations.py \
  --raw_ann merged_cont_A.jsonl \
  --image_root ./ \
  --yolo_model yolov8x.pt \
  --out contA_with_yolo.jsonl \
  --conf 0.25 \
  --iou 0.45 \
  --max_dets 50 \
  --filter_by_question